###### Coursera  | Andrew Ng's Deep Learning Class | Course1. Neural Networks and Deep Learning | Week2. Neural_Networks_Basics

# [1]. Logistic Regression as a Neural Network

신경망 프로그래밍의 기본에 대해서 다시 살펴볼 것이다. 사실 신경망을 구현할 때 몇가지 기술들이 있으며 이것들은 매우 중요하다. 예를들어 $m$건의 훈련 데이터(training data)가 있을 때, for-loop를 이용해 데이터 한건 한건 처리할수도 있다. 하지만 보다 나은 접근은 for-loop 없이 전체 데이터를 한꺼번에 처리하는 것이고, 이를 어떻게 할 수 있는지 살펴볼 것이다. 다음으로 일반적인 신경망을 학습시킬 때 순전파(forward propagation)와 역전파(back propagation)를 수행하는데 왜 이런 것들이 필요한지도 살펴볼 것이다. 이해를 돕기 위해 로지스틱 회귀(logistic regression) 모형을 이용한 예제를 이용할 것이다.

### [1-1]. Binary Classification

로지스틱 회귀는 이항 분류(binary classification) 문제를 풀기 위한 방법론이다. 간단한 이항 분류 문제로 아래 그림과 같이 64 x 64의 이미지가 주어지고 이것이 고양이 그림인지(1) 혹은 그렇지 않은지(0)을 분류해야 한다고 해 보자. (앞으로 출력값을 $y$로 표기하기로 한다.)

(아래 예시 그림은 공간이 부족하여 5 x 5 행렬로 그림) 이 예제의 이미지는 각각 적색, 녹색, 파란색의 강도(0~255)를 표현하는 3개의 64 x 64 행렬로 컴퓨터에 저장된다. 입력 데이터를 열 벡터로 표현해보면 우선 적색 행렬의 좌측 상단부터 255, 231, 42, 22, 123, ... , 202를 아래로 나열하고, 그 아래로 녹색 행렬의 숫자들 255, 134, 202, ... 그리고 파란색 행렬의 숫자를 모두 나열한다. 이는 $\mathbb{R}^{12288 \times 1}$의 열 벡터가 될 것이다. ($64 \times 64 \times 3 = 12288$). 이때 입력 피처(input feature) 크기를 $n_x$로 표기하기로 하며, 아래 예에서는 그 값이 12288이다. (종종 간단히 $n$이라 표기하기도 할 것이다)

결국 이 문제에서 하려는 것은 $x$가 주어졌을 때 $y$를 맞추는 것이다.

![Local image](./images/C1W2L01_01.PNG)

###### 표기법

- 훈련 데이터 한건은 $(x, y)$의 쌍이며, $x$는 $x \in \mathbb{R}^{n_x}$의 열-벡터(column vector)이고, $y$는 1 혹은 0의 값을 갖는다.($y \in \{0, 1\}$)
- 훈련 데이터의 건수는 $m$이라 표기할 것이다. 즉 $\left\{(x^{(1)}, y^{(1)}), ~ (x^{(2)}, y^{(2)}), ~ \cdots ~, (x^{(m)}, y^{(m)})\right\}$로 구성된 데이터이다.
- 종종 훈련 데이터의 건수라는 것을 표현하기 위해 $m_{train}$와 같은 표기도 사용할 것이며, 테스트 데이터의 건수는 $m_{test}$로 표기할 것이다.
- 입력 데이터를 행렬로 표현할 때는 $X$로 표기할 것이며, 각 변수가 열-벡터로 $m$개 열이 $X$을 구성하게 된다. 물론 각 데이터 건을 전치된 열-벡터(transposed column vector)로 위에서부터 쌓는 구조로 $X$를 표현할수도 있으나 각 변수를 열-벡터로 표현하면 그 구현이 훨신 수월해진다.
- 출력 데이터는 $m$건의 출력값을 행-벡터($y \in \mathbb{R}^{1\times m}$)로 표현한다. 물론 열-벡터로 표현하는 것도 가능하지만 행-벡터로 표현하는 것이 파이썬 구현을 훨씬 수월하게 하며, 행-벡터로 표현했을 때 파이썬 벡터의 shape값이 (1, m)이 된다.

![Local image](./images/C1W2L01_02.PNG)

### [1-2]. Logistic Regression

지도학습 특히 타겟 값이 0vs1과 같이 이항인 경우 유용한 방법 중 하나가 로지스틱 회귀 모형이다. 이런 모형을 통해 타겟 값이 0일지 혹은 1일지를 예측하는 것이 목표이며, 보다 엄밀히는 어떤 입력 데이터 $x$가 주어졌을 때 타겟값이 1일 확률값, $\hat{y}=P(y=1 ~|~ x)$를 얻게 되는 것이다.(혹은 0일 확률) 

이때 $x$는 $n_x$차원 벡터($x\in \mathbb{R}^{n_x}$)이고      
모형의 모수(parameter)  $b$와 $w$는 각각 실수($b \in \mathbb{R})$와 $n_x$차원 벡터($w\in \mathbb{R}^{n_x}$)이다.

로지스틱 회귀모형은 $\hat{y}=\sigma(w^Tx + b)$의 형태로 입력과 출력의 관계를 표현한다. 여기서 $\sigma$는 sigmoid 함수를 의미하며 아래와 같이 정의되며 $z$값이 커질수록 1에 가까운 출력값을, 반대로 $z$값이 작아질수록 0에 가까운 출력값을 내놓는다.
$$\sigma(z) = \frac{1}{1 + \exp^{-z}}, ~ z = w^Tx + b$$

지금까지 설명에서 bias 항 $b$를 별도로 표현해 왔으나, $b$를 $\theta_0$으로 표현하고 값이 1인 $x_0$를 추가하여 $\hat{y}=\sigma(\theta^Tx)$와 같이 표현하는 방법도 있다. 우리 강의에서는 bias 항을 $b$로 표현하는 방식을 따를 것이다.

![Local image](./images/C1W2L02_01.PNG)

### [1-3]. Logistic Regression Cost Function

$i$번째 데이터 샘플을 $(x^{(i)}, y^{(i)})$라 할 때 로지스틱 회귀모형의 목표는 모형 출력값인 $\hat{y}^{(i)}$가 실제 타겟 값인 $y^{(i)}$와 최대한 가까워지는 것이다. 보다 일반적으로 전체 데이터 $m$건에 대하여 그 차이가 작다면 좋을 것이다. 이를 위해 일반적으로 많이 사용하는 오차 제곱합을 최소화 하는 접근법($\mathrm{argmin}_{w, b}\sum_i^m \frac{1}{2}\left( \hat{y}^{(i)} - y \right)^2$)을 사용할 수도 있을것 같으나, 최소화 하려는 이 함수가 볼록함수(convex function)가 아니라는 문제가 있다. 이 때문에 다른 형태의 오차(loss)함수와 비용(cost) 함수를 정의하여 사용하게 된다.

우선 오차(loss, error) 함수로 $\mathcal{L}(\hat{y}, y) = \frac{1}{2}\left( \hat{y} - y \right)^2$ 대신, $\mathcal{L}(\hat{y}, y) = -\left( y\log\hat{y} + (1-y) \log(1-\hat{y})\right)$를 사용한다. (※ 샘플 하나에 대한 오차를 loss라 부름) 이 함수의 의미를 생각해보면 $y=1$일 때에는 loss 함수가 $\mathcal{L}(\hat{y}, y) = -\log \hat{y}$가 되며 예측값이 1에서 멀어질 수록 큰 오차를 부과하며, 반대로 $y=0$일 때에는 오차 함수가 $\mathcal{L}(\hat{y}, y) = -\log(1- \hat{y})$가 되며 예측값이 0에서 멀어질 수록 큰 오차를 부과함을 알 수 있다.

결국 전체 $m$건의 데이터에 대한 비용(cost) 함수는 아래와 같다.

$$J(w, b) = \frac{1}{m} \sum_{i=1}^m \mathcal{L}(\hat{y}^{(i)}, y^{(i)}) = - \frac{1}{m} \sum_{i=1}^m \left[ y^{(i)} \log \hat{y}^{(i)} + (1-y^{(i)}) \log(1-\hat{y}^{(i)})\right]$$

![Local image](./images/C1W2L02_02.PNG)

### [1-4]. Gradient Descent

최적화(optimize)할 비용함수 $J(w, b)$를 정의 했었고, 비용함수가 최소값이 되도록 하는 $w$와 $b$를 찾아야 한다. 다행히도 우리가 최적화 하려는 로지스틱 회귀모형의 비용함수는 볼록(convex)함수 이므로 전역 최소값(global minimum)이 단 하나 존재한다. 

전역 최소값을 찾기 위해 우선 $w$와 $b$를 어떤 임의 값으로 초기화 하고, 초기값 $(w, b)$에서 시작하여 가장 경사가 가파른 방향으로 $(w, b)$ 값을 조금식 변화시키는데, 비용함수 값이 임계치 이상 바뀌지 않을 때까지 $(w, b)$ 수정을 반복한다.

![Local image](./images/C1W2L03_01.PNG)

$(w, b)$를 변화시켜가는 과정을 자세히 살펴보자. 설명을 위해 $b$는 없고 $w$만 있다고 가정해보자. 이때 $w$에 따른 비용함수의 값은 아래 그림과 같이 (아래로) 볼록한 함수가 될 것이다. 아래와 같이 $w$ 수정을 반복하며 반복하게 되며 수정의 정도를 표현하기 위해 learning rate $\alpha$를 사용한다.

$$
\begin{align}
&\mathrm{Repeat}  ~ \{ \\
&~~w:=w-\alpha \frac{dJ(w)}{dw} \\
&\} \\
\end{align}
$$

사실 비용 함수에 $w$가 하나만 있지는 않을 것이기 때문에 , 미분을 표현할 때 상미분(ordinary derivative, $dx$)이 아니라 편미분(partial derivative, $\partial x$)으로 표기하는 것이 옳다. 즉 $\frac{\partial J(w, b)}{\partial w_k}$ 로 표기 해야 한다. 

![Local image](./images/C1W2L03_02.PNG)

### [1-5]. Derivatives

$f(a) = 3a$라는 함수가 있을 때 $\frac{df(a)}{da}$는 아주 단순하게 특정 $a$값에서 $a$를 아주 조금 변화시켰을 때 $f(a)$는 얼마나 변하는지에 대한 비율이라 할 수 있다. 예를들어 $a=5$일때 $f(a)=15$이고 $a=5.001$일때 $f(a)=15.003$이다. 즉 $a$가 0.001 증가하면 $f(a)$는 0.003 증가한다. 즉 변화율은 $0.003/0.001 = 3$ 이다. $\frac{df(a)}{da}$의 다른 의미로는 어떤 $a$에서 $f(a)$의 기울기라고 생각할수도 있다.

![Local image](./images/C1W2L04_01.PNG)

### [1-6]. More Derivative Examples

앞서의 예($f(a) = 3a$)의 경우 서로 다른 $a$에서 함수의 기울기가 변하지 않지만, $f(a)=a^2$의 경우 $a$값에 따라 기울기가 달라진다.

![Local image](./images/C1W2L04_02.PNG)

기본적인 몇가지 미분 예는 아래와 같다.
$$
\begin{align}
f(a)=a^2, &~~ \frac{d}{da}f(a) = 2a \\
f(a)=a^3, &~~ \frac{d}{da}f(a) = 3a^2 \\
f(a)=\ln(a), &~~ \frac{d}{da}f(a) = 1/a\\
\end{align}
$$

![Local image](./images/C1W2L04_03.PNG)

### [1-7]. Computation graph

신경망의 연산은 순전파(forward propagation)에 이은 역전파(backward propagation)로 구성된다. Computation Graph를 통해 신경망 연산이 왜 이렇게 구성되어 있는지 살펴볼 것이다.

설명을 위해 아주 간단한 예시 함수 $J(a, b, c)=3(a + bc)$의 순전파와 역전파 과정을 Computation Graph로 살펴보자. 

우선 순전파 과정은 입력값 $(a, b, c)$를 이용해 $J$를 계산하는 것인제 3단계로 나뉜다. 우선 $b$와 $c$를 이용해 $u=bc$를 계산하고, 두번째로 $a$와 $u$를 이용해 $v=a+u$를 계산하고, 마지막으로 $v$를 이용해 $J=3v$를 계산한다. (아래 그림의 파란색 화살표)

![Local image](./images/C1W2L05_01.PNG)

### [1-8]. Derivatives with a Computation Graph

다음으로 역전파(backward propagation)는 $J$에 대한 미분을 구하는 과정인데, Computation Graph로 이 과정을 어떻게 나타낼 수 있는지 살펴본다.

우선 $J=3v$이므로 $\frac{dJ}{dv}$는 $3$이라는 것을 알 수 있다. 두번째로 $\frac{dv}{da}$는 $1$이고, 세번째로 $\frac{dJ}{da}$는 $\frac{dJ}{dv}\frac{dv}{da}$이므로 $3$이고 마찬가지로 $\frac{dJ}{du}=\frac{dJ}{dv}\frac{dv}{du}=3$이다. 

실제 구현에서 대부분의 미분이 $J$를 미분하는 것이므로 앞으로 코드에서는 $\frac{d\mathbf{FinalOutputVar}}{d\mathbf{var}}$에 대한 변수명으로 dvar을 사용할 것이다. 예를들어 $\frac{dJ}{du}$에 대한 변수명으로 'du'를 사용할 것이다.
- dv = 3
- da = 3
- du = 3

![Local image](./images/C1W2L05_02.PNG)

다음으로 $\frac{dJ}{db}=\frac{dJ}{dv}\frac{dv}{db}=3c$인데 이 미분은 $b$가 아닌 다른 변수는 주어진 경우 이므로 그 값이 $6$이다. 마찬가지로 $\frac{dJ}{dc}=\frac{dJ}{dv}\frac{dv}{dc}=3b=9$이다.
- db = 3
- dc = 9

결국 Computation Graph의 역방향을 따라 chain rule을 이어갈 수 있음을 알 수 있다.
(예를들어 $\frac{dJ}{db}=\frac{du}{db}\frac{dv}{du}\frac{dJ}{dv}$이다.)

![Local image](./images/C1W2L05_03.PNG)

### [1-9]. Logistic Regression Gradient Descent

Computation Graph를 이용해 ($w$가 두개인)로지스틱 회귀모형의 비용함수(cost function)에 대한 미분항들을 구해보자.

한건의 데이터 샘플에 대한 오차 함수(loss, error function), $\mathcal{L}(a, y)$를 구하는 과정은 $(x_1, w_1, x_2, w_2, b)$가 주어졌을 때 $z$, $\hat{y}$, $\mathcal{L}(a, y)$를 차례로 구하는 것으로 나눌 수 있고, 이 과정을 아래 그림과 같은 Computation Graph로 나타낼 수 있다.

결국 주어진 데이터를 이용해 최적의 모형을 찾는 것은 $\mathcal{L}(a, y)$가 줄어들도록 $w_1$과 $w_2$를 변화시키는 것이며 이를 위해서는 $\frac{\mathcal{L}(a,y)}{dw_1}$과 $\frac{\mathcal{L}(a,y)}{dw_2}$를 구해야 한다.


![Local image](./images/C1W2L06_01.PNG)

우선 $\mathcal{L}(a,y) = -\left( y\log\hat{y} + (1-y) \log(1-\hat{y})\right)$를 $a$관하여 미분하여 $\frac{\mathcal{L}(a,y)}{da}=-\frac{y}{a} + \frac{1-y}{1-a}$라는 것을 알 수 있고 이것을 'da'라고 표기하기로 한다. 

다음으로 $\frac{d\mathcal{L}(a,y)}{dz}=\frac{d\mathcal{L}(a,y)}{da}\frac{da}{dz} = \left(-\frac{y}{a} + \frac{1-y}{1-a}\right) \cdot a(1-a) = a-y$이고 이것을 'dz'라 표기하기로 한다.

마지막으로  $\frac{d\mathcal{L}(a,y)}{dw_1}=\frac{d\mathcal{L}(a,y)}{da}\frac{da}{dz}\frac{dz}{dw_1} = \frac{d\mathcal{L}(a,y)}{dz}\frac{dz}{dw_1} = \frac{d\mathcal{L}(a,y)}{dz} \cdot x_1$이고 이것을 'dw1'이라 표기할 것이며 $x_1\cdot$'dz'와 같다. 마찬가지로 dw2는 $x_2 \cdot$'dz'이고, 'db'는 $\frac{dz}{db}=1$이므로 'dz'와 같다.($\frac{d\mathcal{L}(a,y)}{db} = \frac{d\mathcal{L}(a,y)}{dz}\frac{dz}{db}$)

Computation Graph에서 이 계산 과정을 살펴보면 $\frac{d\mathcal{L}(a,y)}{dw_1}$을 구하는 것은 단지 (마지막 노드로부터) 노드 [$a=\sigma(z)$]까지 계산된 미분항 'dz'에 $\frac{dz}{dw_1}$을 곱하는 것이다.

![Local image](./images/C1W2L06_02.PNG)

### [1-10]. Gradient Descent on m Examples

$J(w,b)=\frac{1}{m}\sum_{i=1}^m \mathcal{L}(a^{(i)}, y^{(i)})$이므로, 전체 $m$건의 데이터에 대한 비용-미분항들은 아래와 같다.     
$$\begin{align}
\frac{dJ(w,b)}{dw_1}&=\frac{1}{m}\sum_{i=1}^m \frac{d\mathcal{L}(a^{(i)}, y^{(i)})}{dw_1} \\
\frac{dJ(w,b)}{dw_2}&=\frac{1}{m}\sum_{i=1}^m \frac{d\mathcal{L}(a^{(i)}, y^{(i)})}{dw_2} \\
\frac{dJ(w,b)}{db}&=\frac{1}{m}\sum_{i=1}^m \frac{d\mathcal{L}(a^{(i)}, y^{(i)})}{db} \\
\end{align}$$ 

![Local image](./images/C1W2L06_03.PNG)

$J(w,b)$, $\frac{dJ(w,b)}{dw_1}$, $\frac{dJ(w,b)}{dw_2}$, $\frac{dJ(w,b)}{db}$를 구하는 구현 로직은 아래와 같다. 

$m$건에 대한 for-loop를 한번 돌고 나서, 계산된 dw1, dw2, db를 이용해 w1, w2, b를 learning rate $\alpha$에 비례하여 업데이트 하게 된다. 이후 for-loop를 돌고 w1, w2, b를 업데이트 하는 과정을 (J의 변화가 임계값 이하 일때까지)반복한다. 

구현 중간에 'dw1', 'dw2'를 각각 구하는 부분($dw_1 += x_1^{(i)}~dz^{(i)}$, $dw_2 += x_2^{(i)}~dz^{(i)}$)이 있는데 이를 벡터연산 $dw += x^{(i)}$으로 바꿀수도 있다. (이런 벡터연산화는 Feature가 매우 많을 경우 더더욱 효과적일 것이다.)

![Local image](./images/C1W2L06_04.PNG)

# [2]. Python and Vectorization

### [2-1]. Vectorization

벡터 혹은 배열에 대한 연산을 각 원소에 대해 수행하는 것이 아니라, 한번에 명령으로 병렬적으로 동시에 계산하도록 하는 것을 의미한다. 아래 코드에서 확인 할 수 있는 것 처럼 'np.dot(a, b)'는 벡터화되어 연산이 수행되며 그 계산 속도가 벡터화 하지 않은 것에 비해 수십배 빠름을 확인 할 수 있다.

In [50]:
import numpy as np
import time

a = np.random.rand(1000000)
b = np.random.rand(1000000)

tic = time.time()
c = np.dot(a, b)
toc = time.time()

print(c)
print('Vectorized version:', str(1000*(toc-tic)) + 'ms')

c = 0
tic = time.time()
for i in range(100000):
    c += a[i] * b[i]
toc = time.time()

print(c)
print('For loop version: ', str(1000*(toc-tic)) + 'ms')

249961.984627
Vectorized version: 1.0006427764892578ms
25115.3121612
For loop version:  102.07128524780273ms


![Local image](./images/C1W2L07_01.PNG)

### [2-2]. More Vectorization Examples

신경망 프로그래밍시 성능을 위해서 가능하다면 최대한 for-loop 사용을 피하는 것이 좋다. 아래 두가지 예를 통해 for-loop 구현과 벡터화된 연산의 성능 차이를 확인할 수 있다.

m x n차원 행렬 $A$와 m차원 벡터 $v$의 곱을 아래와 같이 for-loop로 계산할수도 있고 벡터화된 np.dot을 이용해 계산할수도 있다.


In [1]:
import numpy as np
import time

n = 2000
m = 3000
A = np.random.random((n, m))
v = np.random.random(m)

u = np.zeros((n,1))

tic = time.time()
for i in range(n):
    for j in range(m):
        u[i] += A[i][j] * v[j]
toc = time.time()
print('For loop version  : ', str(1000*(toc-tic)) + 'ms')

tic = time.time()
u = np.dot(A, v)
toc = time.time()
print('Vectorized version:', str(1000*(toc-tic)) + 'ms')

For loop version :  10045.147180557251ms
Vectorized version: 726.5176773071289ms


m x n차원 행렬 $A$와 n x p차원 행렬 $B$의 곱은 아래와 같다.

In [11]:
import numpy as np
import time

n = 31
m = 71
p = 111

A = np.random.random((n, m))
B = np.random.random((m, p))

u = np.zeros((n,p))

tic = time.time()
for i in range(n):
    for e in range(p):
        for j in range(m):
            u[i][e] += A[i][j] * B[j][e]
toc = time.time()
print('For loop version  : ', str(1000*(toc-tic)) + 'ms')

tic = time.time()
u = np.dot(A, B)
toc = time.time()
print('Vectorized version:', str(1000*(toc-tic)) + 'ms')

For loop version  :  324.2485523223877ms
Vectorized version: 3.991842269897461ms


벡터화된 행렬 연산 몇가지(dot, outer, multiply)

In [96]:
import numpy as np
x = np.random.randn(7)
y = np.random.randn(7)

In [97]:
np.dot(x, y)

-0.10770241043096837

In [98]:
np.outer(x, y)

array([[-0.39572933,  0.25483774,  0.3344601 ,  0.75552084, -0.01264862,
         0.50212055,  0.18370116],
       [ 0.33583878, -0.21627004, -0.28384217, -0.64117865,  0.01073435,
        -0.42612852, -0.15589942],
       [ 0.33544492, -0.2160164 , -0.28350929, -0.6404267 ,  0.01072176,
        -0.42562877, -0.15571659],
       [-0.19284141,  0.12418405,  0.16298453,  0.36817009, -0.00616375,
         0.24468653,  0.08951874],
       [-0.21803739,  0.1404095 ,  0.18427951,  0.41627389, -0.00696909,
         0.2766564 ,  0.10121494],
       [-0.3159111 ,  0.20343721,  0.26699981,  0.60313299, -0.01009741,
         0.4008433 ,  0.14664881],
       [-0.05549641,  0.03573802,  0.04690412,  0.10595297, -0.00177382,
         0.07041654,  0.02576194]])

In [100]:
np.multiply(x, y)  # same as x*y

array([-0.39572933, -0.21627004, -0.28350929,  0.36817009, -0.00696909,
        0.4008433 ,  0.02576194])

![Local image](./images/C1W2L07_03.PNG)

벡터를 인자로 받아 어떤 함수값 벡터를 계산하는 경우도 벡터화 할 경우 그 성능이 매우 향상된다.

numpy의 벡터화된 함수로는 np.exp, np.abs, np.maximum 등이 있고, 피연산자가 numpy-array일 경우 기본적인 4칙연산도 벡터화되어 계산된다. (아래 행렬 제곱에서의 성능 차이 예시)

In [2]:
import numpy as np
import time

n = 1000
m = 2000 
A = np.random.random((n, m))

u = np.zeros((n, m))

tic = time.time()
for i in range(n):
    for j in range(m):
        u[i][j] = A[i][j]**2
toc = time.time()
print('For loop version  : ', str(1000*(toc-tic)) + 'ms')

tic = time.time()
u = A**2
toc = time.time()
print('Vectorized version:', str(1000*(toc-tic)) + 'ms')

For loop version  :  1826.2999057769775ms
Vectorized version: 8.003950119018555ms


![Local image](./images/C1W2L07_04.PNG)

로지스틱 회귀모형의 비용함수(cost function, $J$)와 미분항($dw_1$, $dw_2$, $db$)을 구하는 구현은 아래와 같다. 각 데이터 샘플에 대해 for-loop를 돌게되는데, 우선 dw1과 dw2를 구하는 것을 벡터화 할 수 있다. 지금은 w가 단 두개이지만 여럿일 경우 벡터화 하는 것이 효과적이다. 

즉 d$w_1$ += $x_1^{(i)}$d$z^{(i)}$, d$w_2$ += $x_2^{(i)}$d$z^{(i)}$를 d$w$ += $x^{(i)}$d$z^{(i)}$로 바꾸게 된다.

(아래 슬라이드에서 d$z^{(i)}$ = $a^{(i)} -y^{(i)}$가 d$z^{(i)}$ = $a^{(i)}(1-a^{(i)})$로 표기된 것은 오류로 보임)

![Local image](./images/C1W2L07_05.PNG)

위 슬라이드 내용을 예시 데이터로 구현해보면 아래와 같고, 두번째 코드와 같이 벡터화 할 수 있다. 그런데 $w$가 2개 뿐이라서 벡터화 했을 때 실행시간 은 오히려 증가한다.

In [156]:
# Step1 : implementation with for-loop, 100 epoch

import pandas as pd
import numpy as np
import time

# etc
sigmoid = lambda x: 1 / (1 + np.exp(-x))


def get_log_losses(y_true, y_pred):
    return np.multiply(-y_true, np.log(y_pred)) - np.multiply(1 - y_true, np.log(1 - y_pred))


# data
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)
df[4] = np.where(df[4] == 'Iris-setosa', 1, 0)
df_train = df.sample(frac=0.7, random_state=0)
df_test = df.sample(frac=0.3, random_state=0)

X_test = np.array(df_test[[0, 1]]).T
Y_test = np.array(df_test[[4]]).T
X_train = np.array(df_train[[0, 1]]).T
Y_train = np.array(df_train[[4]]).T

# model parameters
n_x = 2  # number of features
m = X_train.shape[1]  # number of samples
alpha = 0.01  # learning rate
num_epoch = 100

# weights
b = 0
w1 = 0
w2 = 0

w = np.zeros((n_x, 1))

tic = time.time()

# Training - looping for epoch and for all traning data
for _ in range(num_epoch):  # for each epoch

    # initializing gradients and cost 
    J = 0
    dw1 = 0
    dw2 = 0
    db = 0

    for i in range(m):  # for each sample
        x_i_1 = X_train[0, i]  # 1st feature
        x_i_2 = X_train[1, i]  # 2nd feature

        y_i = float(Y_train[0, i])

        z_i = b + w1 * x_i_1 + w2 * x_i_2
        a_i = sigmoid(z_i)
        J += -(y_i * np.log(a_i) + (1 - y_i) * np.log(1 - a_i))

        dz_i = a_i - y_i

        dw1 += x_i_1 * dz_i
        dw2 += x_i_2 * dz_i
        db += dz_i

    J = J / m
    dw1 = dw1 / m
    dw2 = dw2 / m
    db = db / m

    w1 -= alpha * dw1
    w2 -= alpha * dw2
    b -= alpha * db

# Prediction for test set
y_pred = sigmoid(b + w1 * X_test[0, :].reshape(-1, 1) + w2 * X_test[1, :].reshape(-1, 1)).T

# Test set loss
losses = get_log_losses(Y_test, y_pred)

toc = time.time()
print('Average loss for the test set:{}'.format(np.average(losses)))
print('b:{}, w:{}'.format(b, [w1, w2]))
print('Execution time for the loop version  : ', str(1000 * (toc - tic)) + 'ms')

Average loss for the test set:0.5882050769314021
b:0.004373669091718309, w:[-0.22364716321434031, 0.1304488882856519]
Execution time for the loop version  :  76.0345458984375ms


In [157]:
# Step2 : vectorized dw and xi

import pandas as pd
import numpy as np
import time

# etc
sigmoid = lambda x: 1 / (1 + np.exp(-x))


def get_log_losses(y_true, y_pred):
    return np.multiply(-y_true, np.log(y_pred)) - np.multiply(1 - y_true, np.log(1 - y_pred))


# data
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)
df[4] = np.where(df[4] == 'Iris-setosa', 1, 0)
df_train = df.sample(frac=0.7, random_state=0)
df_test = df.sample(frac=0.3, random_state=0)

X_test = np.array(df_test[[0, 1]]).T
Y_test = np.array(df_test[[4]]).T
X_train = np.array(df_train[[0, 1]]).T
Y_train = np.array(df_train[[4]]).T

# model parameters
n_x = X_train.shape[0]  # number of features
m = X_train.shape[1]  # number of samples
alpha = 0.01  # learning rate
num_epoch = 100

# weights
b = np.zeros((1, 1))
w = np.zeros((n_x, 1))  # column vector

tic = time.time()

# Training - looping for epoch and for all traning data
for _ in range(num_epoch):  # for each epoch

    # initializing gradients and cost 
    J = 0;
    dw = np.zeros((n_x, 1))
    db = np.zeros((1, 1))

    for i in range(m):  # for each sample
        x_i = X_train[:, i].reshape(-1, 1)  # column vector
        y_i = Y_train[0, i].reshape(-1, 1)

        z_i = b + np.dot(w.T, x_i)
        a_i = sigmoid(z_i)
        J += -(y_i * np.log(a_i) + (1 - y_i) * np.log(1 - a_i))

        dz_i = a_i - y_i

        dw += np.dot(x_i, dz_i)
        db += dz_i

    J = J / m
    dw = dw / m
    db = db / m
    
    w -= alpha * dw
    b -= alpha * db

# Prediction
y_pred = sigmoid(b + np.dot(w.T, X_test))

# Accuracy
losses = get_log_losses(Y_test, y_pred)

toc = time.time()
print('Average loss for the test set:{}'.format(np.average(losses)))
print('b:{}, w:{}'.format(b.ravel(), list(w.ravel())))
print('Execution time for the vectorized version  : ', str(1000 * (toc - tic)) + 'ms')

Average loss for the test set:0.5882050769314021
b:[ 0.00437367], w:[-0.22364716321434031, 0.13044888828565188]
Execution time for the vectorized version  :  263.1685733795166ms


### [2-3]. Vectorizing Logistic Regression

앞서 구현에서 d$w$ 계산을 벡터화 하기는 했으나 $m$개 샘플에 대한 for-loop가 수행된다. 이 또한 벡터화 할수 있다.

for-loop를 돌게 되면 $m$개의 $z^{(i)}$, $a^{(i)}$를 구하게 되는데 이를 벡터화 하여 $Z = \left[ z^{(1)}, z^{(2)}, \cdots, z^{(m)} \right] = w^TX + b = \left[ w^Tx^{(1)}+b, w^Tx^{(2)}+b, \cdots, w^Tx^{(m)}+b \right]$와 같이 구현할 수 있다. 또한 각 샘플에 대한 예측값 벡터는 $A = \left[ a^{(1)}, a^{(2)}, \cdots, a^{(m)} \right]=\sigma(Z)$=np.exp($Z$)와 같이 구현할 수 있다.

여기에서 $w^TX + b$를 보면 $w \in \mathbb{R}^{n_x \times 1}$, $X \in \mathbb{R}^{n_x \times m}$이라서 '1xm차원 벡터($w^TX \in \mathbb{R}^{1 \times m}$)와 실수($b \in \mathbb{R}$)의 합'이라는 뭔가 부자연스러운 연산이 포함되어 있다. 하지만 + 연산의 첫번째 피연산자가 numpy-array라서 두번째 피연산자인 $b$는 1xm차원 벡터 $[b, b, \cdots, b]$로 확장된다. 이런 것을 Broadcasting이라 한다.

![Local image](./images/C1W2L08_01.PNG)

### [2-4]. Vectorizing Logistic Regression's Gradient Output

앞서 각 샘플 데이터에 대한 예측값 계산을 벡터화하여 벡터 $A$를 구할 수 있었다. dz, db, dw에 대한 연산도 아래와 같이 벡터화 할 수 있다.

![Local image](./images/C1W2L08_02.PNG)

In [154]:
# Step3 : fully vectorized

import pandas as pd
import numpy as np
import time

# etc
sigmoid = lambda x: 1 / (1 + np.exp(-x))
def get_log_losses(y_true, y_pred):
    return np.multiply(-y_true, np.log(y_pred)) - np.multiply(1-y_true, np.log(1-y_pred))

# data
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)
df[4] = np.where(df[4] == 'Iris-setosa', 1, 0)
df_train = df.sample(frac=0.7, random_state=0)
df_test = df.sample(frac=0.3, random_state=0)

X_test = np.array(df_test[[0, 1]]).T
Y_test = np.array(df_test[[4]]).T
X_train = np.array(df_train[[0, 1]]).T
Y_train = np.array(df_train[[4]]).T

# model parameters
n_x = X_train.shape[0]  # number of features
m = X_train.shape[1]  # number of samples
alpha = 0.01  # learning rate
num_epoch = 100

# weights
b = np.zeros((1, 1))
w = np.zeros((n_x, 1))  # column vector

tic = time.time()

# Training - looping for epoch and for all traning data
for _ in range(num_epoch):  # for each epoch
    
    # initializing gradients and cost 
    J = 0;
    dw = np.zeros((n_x, 1))
    db = np.zeros((1, 1))
    
    Z = np.dot(w.T, X_train) + b
    A = sigmoid(Z)
    J = -(Y_train * np.log(A) + (1 - Y_train) * np.log(1-A))
    dz = A - Y_train
    db = np.sum(dz) / m
    dw = np.dot(X_train, dz.T) / m  # equivalent implementations in 2 lines below
    # dw = np.sum(np.multiply(X_train, dz), axis=1) / m
    # dw = np.average(np.multiply(X_train, dz), axis=1)

    w -= alpha * dw
    b -= alpha * db

# Prediction
y_pred = sigmoid(b + np.dot(w.T, X_test))

# Accuracy
losses = get_log_losses(Y_test, y_pred)

toc = time.time()
print('Average loss for the test set:{}'.format(np.average(losses)))
print('b:{}, w:{}'.format(b.ravel(), list(w.ravel())))
print('Execution time for fully vectorized version  : ', str(1000 * (toc - tic)) + 'ms')

Average loss for the test set:0.5882050769314019
b:[ 0.00437367], w:[-0.22364716321434033, 0.1304488882856519]
Execution time for fully vectorized version  :  4.998445510864258ms


아래 슬라이드의 좌측은 원래 for-loop를 이용한 구현이다. 이를 오른쪽과 같이 벡터화 할 수 있었다. 그런데 실제로 파라미터($b$, $\mathbf{w}$)를 구하기 위해서는 비용함수 값이 전역 최소값(global minimum)에 이를 때 까지 ($b$, $\mathbf{w}$) 조정을 계속 해야 한다. 한번 조정하는 과정을 epoch라 하며 이를 여러번 반복하게 된다.(앞서 구현들에서 'for _ in range(num_epoch):  # for each epoch')

![Local image](./images/C1W2L08_03.PNG)

아래와 같이 scikit-learn의 LogisticRegression를 이용해서 모형변수를 구해볼 수도 있다.   
(앞서 구현과 비교해 scikit-learn를 이용한 구현 평균 log-loss가 현저하게 낮은데, 구현도 epoch를 늘려주면 다소 괜찮은 loss를 얻게 된다.)

> - reshape(1, -1) 은 행렬을 행-벡터로 변환
> - reshape(-1, 1) 은 행렬을 열-벡터로 변환
> - ravel()은 행렬을 rank1 배열로 변환

In [155]:
from sklearn import linear_model
from sklearn import metrics

logreg = linear_model.LogisticRegression(penalty='l2', C=100)
logreg.fit(X_train.T, Y_train.ravel())
y_pred = logreg.predict_proba(X_test.T)[:, 1].reshape(1, -1)  # same as below line of code
# y_pred = sigmoid(logreg.intercept_.reshape(-1, 1) + np.dot(logreg.coef_.reshape(-1, 1).T, X_test))

avg_loss = metrics.log_loss(Y_test.ravel(), y_pred.ravel())  # same as below line of code
# avg_loss = np.average(np.multiply(-Y_test, np.log(y_pred)) - np.multiply(1-Y_test, np.log(1-y_pred)))

print('Average loss for the test set:{}'.format(avg_loss))
print('b:{}, w:{}'.format(logreg.intercept_[0], list(logreg.coef_.ravel())))

Average loss for the test set:0.014064427507692585
b:12.561430588667235, w:[-7.4048827705417875, 8.762486265507464]


### [2-5]. Broadcasting in Python

파이썬에서 broadcasting이 어떻게 동작하는지 알아보자. 아래 예시 데이터는 몇가지 식재료 100g에 들어있는 탄수화물(Carb), 단백질(Protein), 지방(Fat)이 몇 g 들어있는지를 나타낸다. 예를들어 사과 100g에는 탄수화물이 56g, 단백질이 1.2g, 지방이 1.8g 들어 있다.

이때 각 식재료 100g에 탄수화물, 단백질, 지방이 각각 몇 %식 들어있는지 계산하려면?

In [1]:
import numpy as np
A = np.array([[56.0, 0.0, 4.4, 68.0], 
             [1.2, 104.0, 52.0, 8.0], 
             [1.8, 135.0, 99.0, 0.9]])

print(A)

[[  56.     0.     4.4   68. ]
 [   1.2  104.    52.     8. ]
 [   1.8  135.    99.     0.9]]


아래 코드에서 axis=0은 위에서 아래로, axis=1은 좌에서 우로 연산하는 것을 의미한다.

In [2]:
cal = A.sum(axis=0)
print(cal)

[  59.   239.   155.4   76.9]


In [3]:
percentage = 100 * A / cal.reshape(1,4)

In [4]:
percentage

array([[ 94.91525424,   0.        ,   2.83140283,  88.42652796],
       [  2.03389831,  43.51464435,  33.46203346,  10.40312094],
       [  3.05084746,  56.48535565,  63.70656371,   1.17035111]])

![Local image](./images/C1W2L09_01.PNG)

'100 * A'에서 100은 자동으로 행렬 A와 같은 차원으로 확장(broadcast)된다. 마찬가지로 2x3 행렬과 1x3행렬을 더하는 경우 1x3 행렬이 2x3 행렬로 확장된다.

![Local image](./images/C1W2L09_02.PNG)

보다 일반적으로 broadcasting이 어떻게 동작하는지 살펴보면, m x n 행렬에 대한 사칙연산으로 두번째 피 연산자가 1 x n 행렬이면 m x n 행렬로 확장되고, m x 1 행렬이면 m x n 행렬로 확장된다. (Matlab이나 Octave로 신경망을 프로그래밍 할 경우 bsxfun이란 것을 사용하는데 여기서도 비슷하게 broadcasting이 동작한다.)

![Local image](./images/C1W2L09_03.PNG)

### [2-6]. A note on python/numpy vectors

파이썬/넘파이에서 broadcasting과 같이 암묵적으로 동작하는 기능들이 아래와 같이 편리할 때도 있지만 

In [14]:
k = np.array([1, 2, 3])
k + 100

array([101, 102, 103])

아래 예와 같이 의도하지 않은 결과를 줄때도 있다. ((1,3)이나 (3,1)벡터 결과를 의도한 경우)

In [17]:
x = np.array([1, 2, 3]).reshape((3,1))
y = np.array([4, 5, 6]).reshape((1,3))
z = x + y
z

array([[5, 6, 7],
       [6, 7, 8],
       [7, 8, 9]])

이렇게 암묵적 동작 때문에 정말 찾기 어려운 버그가 발생하기도 하는데, 이런 문제들을 피하기 위한 팁 몇가지를 알아보기로 한다. 우선 정규분포에서 임의의 숫자 5개를 생성한다.

In [19]:
import numpy as np

a = np.random.randn(5)
print(a)

[-0.34874007  2.68509097  0.15072409 -0.57173937  1.90800882]


이 때 변수 a의 shape은 (5,)로서 이건 열벡터도, 행벡터도 아닌 랭크1-배열(rank1 array)이라는 것이다.

In [46]:
print(a.shape)  # This is rank-1 array, neither row vector or column vector

(5,)


문제는 a를 열벡터 혹은 행벡터로 생각하고 전치(transpose)를 하더라도 그 shape이 바뀌지 않는다.

In [47]:
print(a.T)

[-0.34874007  2.68509097  0.15072409 -0.57173937  1.90800882]


또한 a와 a를 전치한 것을 dot-product할 경우, a가 벡터라면 어떤 행렬이 나오겠지만 a가 '랭크1-배열'이므로 실수(real number)가 결과 값이 된다.

In [37]:
# Rank1-array Times Rank1-array transposed 
print(np.dot(a, a.T))

# Vector Times Vector transposed
b = np.array([[1], [2], [3]])
print(np.dot(b, b.T))

11.3214344809
[[1 2 3]
 [2 4 6]
 [3 6 9]]


이런 문제를 피하기 위해서 신경망을 코딩할 때에는 shape이 (?, )혹은 (, ?)와 같은 **'랭크1-배열' 사용을 피하라**는 것이다. 대신 어떤 행렬을 생성할 때에는 명시적으로 shape을 지정하는 것이 실수를 줄이는 길이다.

In [43]:
c = np.random.randn(5, 1)
print('c:', c, '\n')
print("c's shape:", c.shape, '\n')
print('c transposed:', c.T)

c: [[ 1.83831117]
 [-2.27910181]
 [ 0.49413789]
 [-0.75069614]
 [ 0.83271093]] 

c's shape: (5, 1) 

c transposed: [[ 1.83831117 -2.27910181  0.49413789 -0.75069614  0.83271093]]


위의 열벡터 c에서 확인 할수 있는 것과 같이 꺽음괄호([, ])가 2중으로 데이터를 감싸고 있음을 확인 할 수 있다.(랭크1-행렬의 경우 꺽음괄호가 한번 감싸고 있다.)

요약하면...    

- 아래와 같은 변수 생성을 지양
```Python
a = np.random.randn(5)
```

- 열벡터를 생성할 때에는 아래와 같은 방식을 사용
```Python
a = np.random.randn(5, 1)
```

- 행벡터를 생성할 때에는 아래와 같은 방식을 사용
```Python
a = np.random.randn(1, 5)
```

- 혹은 변수 생성후 shape 변경(reshape은 CPU 비용이 저렴함)
```Python
a = np.random.randn(5).reshape((1, 5))
```

- assertion을 삽입하여 에러를 감지
```Python
assert(a.shape == (5,1))
```

assertion을 넣으면 아래와 같이 조건(shape == (5,1))에 맞지 않는 경우 알려준다.

In [53]:
a = np.random.randn(5)
assert(a.shape == (5,1))

AssertionError: 

numpy행렬간 연산에서 기본적인 사칙연산(+, -, *, /)은 같은 위치의 원소간(element-wise) 연산이 이뤄지며, (shape의 값이 하나만 다른 경우) broadcast가 일어난다.

아래 예와 같이 '*'와 np.dot의 결과는 전혀 다르다.

In [76]:
a = np.random.randn(2, 3, 4)
b = np.random.randn(1, 3, 4)
a * b

array([[[-0.60645678,  0.14319223,  0.14611721, -0.20050345],
        [-0.11662699,  0.35041491, -0.01819147, -0.89612952],
        [ 0.07840696,  0.73099772,  0.19418043, -1.11964412]],

       [[-0.2158664 ,  0.79288889,  0.58701581,  0.07476491],
        [-0.32170461,  0.1912308 ,  0.02257422,  3.51463411],
        [-0.16082483, -0.6946641 , -0.03444864, -0.57286972]]])

In [77]:
a = np.random.randn(2, 3)
b = np.random.randn(3, 2)
a * b

ValueError: operands could not be broadcast together with shapes (2,3) (3,2) 

In [78]:
a = np.random.randn(2, 3)
b = np.random.randn(3, 2)
np.dot(a, b)

array([[ 1.69071795, -0.13511543],
       [ 4.86600855, -0.01294056]])

### [2-7]. Quick tour of Jupyter/iPython Notebooks

Programming Assignment가 Jupyter notebook 형태로 제공될텐데, 이에 대한 사용법을 설명함.

- '### START CODE HERE ###'과 '### END CODE HERE ###' 사이에 과제 코드 작성
- 코드 셀 실행은 shift + enter
- 작성이 끝나면 우측 상단에 'Submit Assigment' 버튼으로 제출

### [2-8]. Explanation of logistic regression cost function (optional)

일반적인 회귀모형에서는 예측값과 실제값의 차이에 대한 제곱합을 비용함수로 사용하는데($J(b, w) = - \frac{1}{m} \sum_i^m (y^{(i)} - \hat{y}^{(i)})^2$), 앞서 설명한 로지스틱 회귀모형에서는 다른 형태의 비용함수($J(w, b) = - \frac{1}{m} \sum_{i=1}^m \left[ y^{(i)} \log \hat{y}^{(i)} + (1-y^{(i)}) \log(1-\hat{y}^{(i)})\right]$)를 사용 했었다. 왜 이렇게 하는 것인지 알아보자.

로지스틱 회귀모형에 의하면 어떤 데이터 1건 $\mathbf{x}$가 가 주어졌을 때, 실제값이 1일 확률은 $p(y=1|\mathbf{x}) = \hat{y}$이고, 실제값이 0일 확률은 $p(y=0|\mathbf{x}) = 1- \hat{y}$이다. 이 두 표현을 결합하여 실제값이 1이든지 0이든지 그것을 맞출 확률은 $p(y|\mathbf{x}) = \hat{y}^y (1-\hat{y})^{1-y}$이다. 전체 $m$건의 데이터가 주어졌을 때 전체 데이터의 실제값을 맞출 확률은 각 건을 맞출 확률을 모두 곱한(결합확률, iid를 가정함) $\prod_i^m p(y^{(i)}|\mathbf{x}^{(i)})^{(i)}$이 된다.(이것을 우도(likelihood) 라 한다.)

즉 $\prod_i^m p(y^{(i)}|\mathbf{x}^{(i)})^{(i)}$값이 크다면 좋은 모형이라 할 수 있고,  이 값이 가장 커지도록 하는 적절한 $(b, \mathbf{w})$를 찾는 것이 모형 학습 과정이라 할 수 있다. (물론 과적합을 피하기도 해야 한다.)

그런데 log 함수의 단조증가 특성 때문에 $\prod_i^m p(y^{(i)}|\mathbf{x}^{(i)})^{(i)}$를 최대화 하는 $(b, \mathbf{w})$과 
log를 취하고 $m$으로 나눈 $\frac{1}{m}\sum_i^m \log p(y^{(i)}|\mathbf{x}^{(i)})^{(i)}$를 최대화 하는 $(b, \mathbf{w})$이 같은데, under-flow(0과 1 사이의 값을 여러번 곱할 경우 그 값이 너무 0에 가까워져서 컴퓨터가 소수점에서 먼 값들을 잃어 버리는 현상) 문제와 계산의 편의성을 위해 log를 취한 값을 최대화하는 $(b, \mathbf{w})$를 찾는다. (이런 방법을 최대 우도 추정법(maximum likelihood estimation)이라 한다. $m$으로 나누는 것은 데이터 건수가 다를 경우에도 비용함수값에 대한 비교를 용이하기 하기 위한 단위 맞춤이다.)

$$\begin{align}
J(b, \mathbf{w}) & =  \frac{1}{m} \log \prod_i^m p(y^{(i)}|\mathbf{x}^{(i)})^{(i)} \\
& = \frac{1}{m}\sum_i^m \log p(y^{(i)}|\mathbf{x}^{(i)})^{(i)} \\
& = \frac{1}{m} \sum_i^m \log \left( \hat{y^{(i)}}^{y^{(i)}} (1-\hat{y^{(i)}})^{1-y^{^{(i)}}}  \right) \\
& = \frac{1}{m} \sum_i^m y^{(i)} \log \hat{y^{(i)}} + (1-y^{(i)}) \log(1-\hat{y^{(i)}})
\end{align}$$

.